<!-- Warning (yellow) -->
<div style="background-color: #fff3cd; border-left: 4px solid #ffc107; padding: 12px; margin: 10px 0;">
<strong>⚠️ Warning</strong><br>
Your warning text here.
</div>

<!-- Success/Tip (green) -->
<div style="background-color: #d4edda; border-left: 4px solid #28a745; padding: 12px; margin: 10px 0;">
<strong>💡 Tip</strong><br>
Your tip text here.
</div>

<!-- Danger/Important (red) -->
<div style="background-color: #f8d7da; border-left: 4px solid #dc3545; padding: 12px; margin: 10px 0;">
<strong>🚨 Important</strong><br>
Critical information here.
</div>

# XNAT & XNATpy Best Practices Workbook

This notebook demonstrates recommended practices for using `xnatpy` to interact with your XNAT :

#### Topics Covered
1. Uploading data to XNAT
2. Exploring and understanding the XNAT data hierarchy
3. Filtering available data based on DICOM metadata
4. Downloading filtered data for local processing


<div style="background-color: #e7f3fe; border-left: 4px solid #2196F3; padding: 12px; margin: 10px 0;">
<strong>📌 Prerequisites</strong><br>
Before starting, ensure you have:
<ul>
<li>A reachable XNAT server URL</li>
<li>Credentials with appropriate project permissions</li>
<li><code>xnat</code> (xnatpy) and optionally <code>pydicom</code> installed</li>
</ul>
</div>


## 0. Session Setup: Connecting to XNAT with xnatpy

In the following cell we have created a reusable helper function to create a connection to our XNAT instance.

<!-- Success/Tip (green) -->
<div style="background-color: #d4edda; border-left: 4px solid #28a745; padding: 12px; margin: 10px 0;">
<strong>💡 Tip</strong><br>
XNAT's Jupyter notebooks come pre-configured with your user credentials as Env variables!
    
  - `XNAT_HOST`
  - `XNAT_USER`
  - `XNAT_PASS`
</div>



In [1]:
import os
import xnat  # Our xnatpy import
from pathlib import Path
from typing import Optional

def get_xnat_session(
    host: Optional[str] = None,
    user: Optional[str] = None,
    password: Optional[str] = None,
):
    """Establish & return an XNAT connection.

    Defaults to environment variables if args not supplied.
    """
    host = host or os.environ.get("XNAT_HOST")
    user = user or os.environ.get("XNAT_USER")
    password = password or os.environ.get("XNAT_PASS")

    if host is None or user is None or password is None:
        raise ValueError(
            "XNAT_HOST, XNAT_USER, XNAT_PASS must be set in the environment or "
            "passed explicitly to get_xnat_session()."
        )

    session = xnat.connect(host, user=user, password=password)
    return session

### Confirming Our Connection
<!-- Warning (yellow) -->
<div style="background-color: #fff3cd; border-left: 4px solid #ffc107; padding: 12px; margin: 10px 0;">
<strong>⚠️ Warning</strong><br>
If you notice a redirect warning, that is okay!
    
"[WARNING] Detected a redirect from..."
</div>

In [2]:
# Taking a look at the available projects is an easy means to check your connection!
with get_xnat_session() as session:
    for project in session.projects: 
        print(f"Project ID: {project}")

[WARNING] Detected a redirect from https://adapt-dev.embarklabs.ai to https://adapt-dev.embarklabs.ai:443/, using https://adapt-dev.embarklabs.ai:443/ from now on
[WARNING] Found an unsupported version (1.9.2.3), trying 1.7 compatible model builder


Project ID: 00001
Project ID: CMB-BRCA
Project ID: NLST
Project ID: NSCLC_RADIOGENOMICS
Project ID: NSCLC_RADIOMICS
Project ID: NSCLC-INTEROBSERVER1
Project ID: RIDER-LUNG-CT
Project ID: NewUserTrainingProject


## 1. Ingesting Data into XNAT

There are two robust options for ingesting imaging data into XNAT:

1. Create a ZIP archive containing a valid XNAT image session hierarchy, or
2. Use a ZIP of raw DICOM files that the XNAT import service can interpret.

### Visualization of the Archive Structure

<div style="background-color: #e7f3fe; border-left: 4px solid #2196F3; padding: 12px; margin: 10px 0;">
<strong>📁 Archive Structure</strong><br>
<strong>Pre-structured Archive</strong>
<pre style="margin: 0; background: none; font-family: monospace;">
archive.zip
├── subject_label/
│   └── session_label/
│       └── scans/
│           ├── 1-T1_MPRAGE/
│           │   └── resources/
│           │       └── DICOM/
│           │           ├── 0001.dcm
│           │           ├── 0002.dcm
│           │           └── ...
│           └── 2-FLAIR/
│               └── resources/
│                   └── DICOM/
│                       └── ...

<em>Stuctured archives give you direct control for using metadata as labels!</em>
</pre>
<br>
<strong>Auto-organized Archive (Raw DICOMs)</strong>
<pre style="margin: 0; background: none; font-family: monospace;">
upload.zip
├── MR.1.2.840.113619.2.416.12345.dcm
├── MR.1.2.840.113619.2.416.12346.dcm
├── MR.1.2.840.113619.2.416.12347.dcm
├── CT.1.2.840.113619.2.312.98765.dcm
├── CT.1.2.840.113619.2.312.98766.dcm
└── ...
</pre>
<br>
<em>Auto-organizing depends on DICOM headers (SeriesInstanceUID, Modality, etc.) to automatically organizes files into the appropriate subject/session/scan structure.</em>
</div>

<!-- Success/Tip (green) -->
<div style="background-color: #d4edda; border-left: 4px solid #28a745; padding: 12px; margin: 10px 0;">
<strong>💡 Tip</strong><br>
Labels from auto-organizing can be messy and unclear!<br>
We strongly recommend <strong>Option 1</strong> to ensure proper and effecitve labeling.
</div>

### 1.1 Example Data Ingest

This example workbook includes sample MR DICOM data in the `example_subject` directory.

This section demonstrates how to structure that DICOM data into the proper XNAT archive format and upload it to your project!

In [3]:
# We are going to explicitly designate our Project/Subj/Path for clarity!
XNAT_PROJECT_ID = "NewUserTrainingProject"   # project ID, not label
XNAT_SUBJECT_LABEL = "Test_Subject_999"  # Make sure to create a new subject label! Add "" once changed!
LOCAL_ZIP_PATH = Path("data") / f"{XNAT_SUBJECT_LABEL}_session.zip"  # path local zip

print("Local upload archive:", LOCAL_ZIP_PATH)

Local upload archive: data/Test_Subject_999_session.zip


### Organizing & Zipping our Data

In [ ]:
from src.utils.structuring import structure_dicom_directory_for_xnat

archive_zip = structure_dicom_directory_for_xnat(
    dicom_dir="example_subject",
    subject_label="Test_Subject_999",
    session_label="Test_Subject_999_MR1",
    output_dir="data",
    create_zip=True,
)

### Uploading our Archive to XNAT

In [ ]:
from src.utils.upload import upload_archive_to_xnat

# Upload our structured archive directly to the XNAT archive (skipping prearchive)
result = upload_archive_to_xnat(
    archive_path=archive_zip,
    project_id=XNAT_PROJECT_ID,
    subject_label=XNAT_SUBJECT_LABEL,
    destination="/archive",  # Skip prearchive, upload directly to archive
)

if result["success"]:
    print(f"Upload successful: {result['message']}")
else:
    print(f"Upload failed: {result['message']}")

## 2. Explore the XNAT data structure via DICOM metadata

XNAT organizes imaging data as:

- Project → Subjects → Experiments/Sessions → Scans → Resources → Files

This section demonstrates how to:

- Extract DICOM metadata directly from files mounted on disk
- Build a DataFrame for exploration and filtering
- No API calls required when data is mounted locally

<!-- Success/Tip (green) -->
<div style="background-color: #d4edda; border-left: 4px solid #28a745; padding: 12px; margin: 10px 0;">
<strong>💡 Tip</strong><br>
Reading metadata directly from DICOM files is faster and more reliable than API calls, especially for large datasets. The XNAT path structure is parsed automatically to extract project, experiment, and scan identifiers.
</div>

In [ ]:
from src.utils.dataframe import collect_dicom_metadata

# Path to the XNAT project data on disk
# This assumes data is mounted at /data/projects/<PROJECT_ID>
XNAT_DICOM_ROOT = f"/data/projects/{XNAT_PROJECT_ID}"

# Build a metadata DataFrame directly from DICOM files on disk
# No API calls required - reads directly from the mounted XNAT archive
df = collect_dicom_metadata(
    root_dir=XNAT_DICOM_ROOT,
    extra_tags=[],      # Add extra DICOM attribute names if needed
    max_files=None,     # Set a limit for sampling; None = all files
)

print(f"Metadata rows: {len(df)}")
df.head()

## 3. Explore and filter data with the interactive dashboard

The dashboard below provides an interactive interface for exploring your DICOM metadata:

**Features:**
- **Toggle filters**: Click checkboxes to filter by Modality, Body Part, Institution, etc.
- **Text search**: Search for specific series descriptions
- **Distribution charts**: Visualize value distributions with bar charts
- **Summary statistics**: See counts of files, experiments, and modalities
- **Export**: Download filtered data as CSV for further analysis

<!-- Success/Tip (green) -->
<div style="background-color: #d4edda; border-left: 4px solid #28a745; padding: 12px; margin: 10px 0;">
<strong>💡 Tip</strong><br>
The charts and statistics update automatically as you change filters. Use this to quickly identify subsets of data for your ML training pipeline.
</div>

In [ ]:
from src.utils.filter_widget import create_metadata_dashboard

# Create an interactive dashboard to explore and filter the DataFrame
# Toggle buttons for categorical fields, range sliders for numeric fields
create_metadata_dashboard(df)

In [ ]:
# 3.1 Example: filter scans based on metadata

import pandas as pd

def filter_scans_by_criteria(df: pd.DataFrame):
    """Example filter: only T1 MR scans (SeriesDescription contains 'T1' and Modality is 'MR')."""
    if df.empty:
        return df

    df_filtered = df.copy()

    # Example filter: modify to match your site/protocol
    # Note: Column names match DICOM tag names (e.g., SeriesDescription, Modality)
    mask = (
        df_filtered["SeriesDescription"].str.contains("T1", case=False, na=False)
        & df_filtered["Modality"].str.upper().eq("MR")
    )

    filtered = df_filtered[mask].reset_index(drop=True)
    return filtered

# Example usage:
# filtered_scans = filter_scans_by_criteria(df)
# display(filtered_scans.head())
# print("Number of filtered scans:", len(filtered_scans))

## 4. Download filtered data locally

Once you have a filtered subset of scans (by DICOM metadata or any other criteria),
you typically want to download them for local processing.

xnatpy provides helper methods:

- `scan.download(path_to_zip)` – download a single scan as a ZIP of all DICOMs
- `subject.download_dir(target_dir)` – download entire subject tree
- `experiment.download_dir(target_dir)` – download entire session

Here we show how to:

- Loop over a filtered list of scan rows
- Resolve the corresponding scan objects
- Download each scan into a per-scan directory


In [ ]:
def download_filtered_scans(
    session: xnat.XNATSession,
    filtered_scans_df: pd.DataFrame,
    base_download_dir: Path,
    project_id: str,
    as_zip: bool = False,
):
    """Download filtered scans to a local directory.

    Parameters
    ----------
    session : xnat.XNATSession
        Active XNAT session.
    filtered_scans_df : pd.DataFrame
        DataFrame produced by the filtering step. Expected columns:
        'xnat_experiment_label', 'xnat_scan_id', and 'PatientID'.
    base_download_dir : Path
        Local directory where data will be saved.
    project_id : str
        XNAT project ID.
    as_zip : bool
        If True, use scan.download() (zip file). If False, use scan.download_dir().
    """
    base_download_dir.mkdir(parents=True, exist_ok=True)
    project = session.projects[project_id]

    for _, row in filtered_scans_df.iterrows():
        # Use the XNAT path info extracted from DICOM file paths
        patient_id = row.get("PatientID", "unknown")
        exp_label = row["xnat_experiment_label"]
        scan_id = str(row["xnat_scan_id"])

        # Navigate XNAT hierarchy to find the scan
        # Note: We need to find the subject that contains this experiment
        experiment = None
        for subject in project.subjects.values():
            if exp_label in subject.experiments:
                experiment = subject.experiments[exp_label]
                break
        
        if experiment is None:
            print(f"Warning: Could not find experiment {exp_label}")
            continue
            
        scan = experiment.scans[scan_id]

        scan_dir = base_download_dir / patient_id / exp_label / scan_id
        scan_dir.mkdir(parents=True, exist_ok=True)

        if as_zip:
            zip_path = scan_dir / f"{scan_id}.zip"
            print(f"Downloading scan {scan_id} as ZIP → {zip_path}")
            scan.download(str(zip_path))
        else:
            print(f"Downloading scan {scan_id} into directory → {scan_dir}")
            scan.download_dir(str(scan_dir))

# Example usage (uncomment when connected and filtered_scans is defined):
# download_root = Path("downloads") / XNAT_PROJECT_ID
# with get_xnat_session() as session:
#     download_filtered_scans(session, filtered_scans, download_root, XNAT_PROJECT_ID, as_zip=False)

---

### Summary

This notebook provides a practical workflow for:

- Uploading DICOM data into XNAT using the import service
- Exploring the project/subject/experiment/scan hierarchy
- Building a metadata table that includes selected DICOM header fields
- Filtering scans based on DICOM metadata criteria
- Downloading only the filtered subset for local processing

You can now adapt the filtering logic to match specific clinical protocols
(e.g. BI-RADS screening MR, specific echo times, body parts, or sequences)
and wire this into your downstream ML training pipeline.
